In [1]:
import folium
import geopandas as gp
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon, Point
pd.set_option('display.max_columns', None)

lat = -14.55353736
long = -56.38679393

m = folium.Map(location=[lat,long], zoom_start=5)
folium.LatLngPopup().add_to(m)
corFloresta = {'fillColor': 'green', 'color': 'green'}


def convert_3D_2D(geometry):
    '''
    Takes a GeoSeries of 3D Multi/Polygons (has_z) and returns a list of 2D Multi/Polygons
    '''
    new_geo = []
    for p in geometry:
        if p.has_z:
            if p.geom_type == 'Polygon':
                lines = [xy[:2] for xy in list(p.exterior.coords)]
                new_p = Polygon(lines)
                new_geo.append(new_p)
            elif p.geom_type == 'MultiPolygon':
                new_multi_p = []
                for ap in p:
                    lines = [xy[:2] for xy in list(ap.exterior.coords)]
                    new_p = Polygon(lines)
                    new_multi_p.append(new_p)
                new_geo.append(MultiPolygon(new_multi_p))
    return new_geo

def filter(geometry,ponto):
    for poly in geometry:
            if poly.geom_type == 'Polygon':
               if ponto.within(poly) or poly.within(ponto):
                    mask = poly  
                    return mask             
            elif poly.geom_type == 'MultiPolygon':
                for p in poly:
                    if ponto.within(p) or p.within(ponto):
                        mask = poly
                        return mask 
    return False


florestas = gp.read_file('dados/florestas/MT_floresta_SireneJud.shp')
florestas.geometry = convert_3D_2D(florestas.geometry)
municipios = gp.read_file('dados/municipios/MT_Municipios_2020.shp')   
sigef = gp.read_file('dados/sigef/Sigef_Brasil_MT.shp')
area_imovel = gp.read_file('dados/area/CarSireneDiamantino.shp')
terra_indigena = gp.read_file('dados/terra_indigena/ti_sirgas.shp')
app = gp.read_file('dados/app/APP.shp')
app_alto = gp.read_file('dados/app_altoParaguai/APP.shp')


ponto =Point(long,lat) 
# folium.Marker([lat,long],icon=folium.Icon(color='red', icon='info-sign' ),popup="Referencia").add_to(m)
# folium.GeoJson(data=florestas["geometry"],style_function=lambda x:corFloresta).add_to(m)

In [29]:
mask_floresta = filter(florestas['geometry'],ponto)
mask_cidade = filter(municipios['geometry'],ponto)
mask_area_imovel = filter(area_imovel['geometry'],ponto)


if mask_floresta: 
    geo_floresta =florestas[florestas['geometry']==mask_floresta] 
    folium.GeoJson(data=geo_floresta["geometry"],style_function=lambda x:corFloresta).add_to(m)
    folium.Marker([lat,long],icon=folium.Icon(color='red', icon='info-sign' ),popup=geo_floresta['SireneJud'].values[0]).add_to(m)

if mask_cidade:
    geo_cidade = municipios[municipios['geometry']==mask_cidade] 
    folium.GeoJson(data=geo_cidade["geometry"]).add_to(m) 
if mask_area_imovel:
    style3 = {'fillColor': '#9c5010', 'color': 'orange'}
    geo_area_imovel = area_imovel[area_imovel['geometry']==mask_area_imovel]
    folium.Marker([lat,long],icon=folium.Icon(color='orange', icon='info-sign' )
                  ,popup=geo_area_imovel['COD_IMOVEL'].values[0]).add_to(m)
    
    folium.GeoJson(data=geo_area_imovel["geometry"],style_function=lambda x:style3).add_to(m)  
    resp = gp.overlay(florestas,geo_area_imovel,how='intersection')   
    folium.GeoJson(data=resp["geometry"],style_function=lambda x:corFloresta).add_to(m)
    x,y = resp['geometry'][0].exterior.coords.xy
    la =y[9] 
    lon = x[9]
    folium.Marker([la,lon],icon=folium.Icon(color='red', icon='info-sign' )).add_to(m)
    p =Point(lon,la)
    mask_floresta = filter(florestas['geometry'],p)
    
    if mask_floresta: 
        geo_floresta =florestas[florestas['geometry']==mask_floresta] 
        folium.GeoJson(data=geo_floresta["geometry"],style_function=lambda x:corFloresta).add_to(m)
        folium.Marker([la,lon],icon=folium.Icon(color='red', icon='info-sign' ),popup=geo_floresta['SireneJud'].values[0]).add_to(m)

In [47]:
resp['geometry'] 

0    POLYGON ((-56.38799 -14.56525, -56.38805 -14.5...
Name: geometry, dtype: geometry

In [44]:
m

In [5]:
mask_floresta


False

In [45]:
y[9]

-14.564630677413025

In [46]:
'''
[-14.565236473999958, -14.565069117999933, -14.564889336999947, -14.564211475999974, 
-14.56423858999995, -14.564455503999971, -14.564509737999973, -14.5645108374914, 
-14.564630677413025, -14.56525047523309, -14.565236473999958]
'''

'\n[-14.565236473999958, -14.565069117999933, -14.564889336999947, -14.564211475999974, \n-14.56423858999995, -14.564455503999971, -14.564509737999973, -14.5645108374914, \n-14.564630677413025, -14.56525047523309, -14.565236473999958]\n'